In [ ]:
!pip install Pipeline

# Home credit

In [ ]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder,OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import time
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report,roc_curve, auc,precision_recall_curve,f1_score,balanced_accuracy_score

In [ ]:
train_path = "/mnt/01D6B57CFBE4DB20/1.Linux/Data/home-credit-default-risk/application_train.csv"
test_path = "/mnt/01D6B57CFBE4DB20/1.Linux/Data/home-credit-default-risk/application_test.csv"
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

In [ ]:
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
                               ...   
AMT_REQ_CREDIT_BUREAU_DAY     float64
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
Length: 122, dtype: object

In [ ]:
# Learn label encoder
le = LabelEncoder()
le.fit(["paris", "paris", "tokyo", "amsterdam"])
print(le)
print(list(le.classes_))

print(le.transform(["tokyo", "tokyo", "paris"]))

print(list(le.inverse_transform([2, 2, 1])))

LabelEncoder()
['amsterdam', 'paris', 'tokyo']
[2 2 1]
['tokyo', 'tokyo', 'paris']


In [ ]:
#train["NAME_CONTRACT_TYPE"]=LabelEncoder().fit_transform(train["NAME_CONTRACT_TYPE"])
#train["NAME_CONTRACT_TYPE"]

0         0
1         0
2         1
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    0
307510    0
Name: NAME_CONTRACT_TYPE, Length: 307511, dtype: int64

In [ ]:
train["NAME_CONTRACT_TYPE"].values

array(['Cash loans', 'Cash loans', 'Revolving loans', ..., 'Cash loans',
       'Cash loans', 'Cash loans'], dtype=object)

In [ ]:
a=str(train["NAME_CONTRACT_TYPE"].values)
a

"['Cash loans' 'Cash loans' 'Revolving loans' ... 'Cash loans' 'Cash loans'\n 'Cash loans']"

In [ ]:
categorical_features = train.select_dtypes(include=['object']).columns
categorical_features

Index(['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE',
       'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE',
       'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE'],
      dtype='object')

In [ ]:
for feature in categorical_features:
    #train[feature]=pd.Categorical(train[feature])
    train[feature]=str(train[feature].values)
    train[feature]=LabelEncoder().fit_transform(train[feature])


In [ ]:
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,0,0,0,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,0,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,0,0,0,0,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,0,0,0,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,0,0,0,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0,0,0,0,0,157500.0,254700.0,27558.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,0,0,0,0,0,72000.0,269550.0,12001.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,0,0,0,0,0,153000.0,677664.0,29979.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,1,0,0,0,0,0,171000.0,370107.0,20205.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
train.dtypes

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE              int64
CODE_GENDER                     int64
FLAG_OWN_CAR                    int64
                               ...   
AMT_REQ_CREDIT_BUREAU_DAY     float64
AMT_REQ_CREDIT_BUREAU_WEEK    float64
AMT_REQ_CREDIT_BUREAU_MON     float64
AMT_REQ_CREDIT_BUREAU_QRT     float64
AMT_REQ_CREDIT_BUREAU_YEAR    float64
Length: 122, dtype: object

In [ ]:
dataX = train.drop(['TARGET'],axis=1)
dataY = train['TARGET']

X_train, X_test, y_train, y_test = \
    train_test_split(dataX, dataY, test_size=0.30, \
                    random_state=2018, stratify=dataY)

In [ ]:
#Standard scaler all numeric value
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns

#One hot encoder all category value
categorical_transformer = Pipeline(steps=[('onehot', OrdinalEncoder())])
categorical_features = X_train.select_dtypes(include=['object']).columns

transformers=[
    ('numeric', numeric_transformer, numeric_features),
    ('categoric', categorical_transformer, categorical_features),
    #('numeric', numeric_transformer, selector(dtype_exclude="object")),
    #('categoric', categorical_transformer, selector(dtype_include="object")),
    #('imputer', imputer_transformer, numeric_features),
]
preprocessor = ColumnTransformer(transformers)

In [ ]:
# Create transform X value
X_train_tran = preprocessor.fit_transform(X_train)
X_test_tran=preprocessor.transform(X_test)

In [ ]:
start=time.time()
xgb=XGBClassifier(
                nthread=16,
                objective='binary:logistic',
                eval_metric = 'logloss',
                #scale_pos_weight=1,
                enable_categorical=True,
                seed=2018,
                verbosity=1    
                              )
xgb.fit(X_train, y_train)
end=time.time()
print("time: ",end-start) # 95 s compare with 108 s h2o r
y_pred=xgb.predict(X_test)
y_pred_probs=xgb.predict_proba(X_test)
print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print('classification_report \n',classification_report(y_test, y_pred))

/home/dnn/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[08:21:54] WARNING: ../src/learner.cc:541: 
Parameters: { enable_categorical } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


95.6757915019989
f1 score 0.04768583450210379
balanced_accuracy_score: 0.5113273773669774
classification_report 
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     84806
           1       0.47      0.03      0.05      7448

    accuracy                           0.92     92254
   macro avg       0.70      0.51      0.50     92254
weighted avg       0.88      0.92      0.88     92254



In [ ]:
start=time.time()
lgbm=LGBMClassifier(
    n_estimators = 200,
    boosting_type = 'gbdt',
    objective = 'binary',
    metric= 'binary:logloss',
    metric_freq=50,
    learning_rate=0.01,
    max_depth=4,
    num_leaves= 16, #(2^maxdepth)
    min_data_in_leaf=2000,
    scale_pos_weight=1,
    num_threads=16,
    random_state =500,
    bagging_freq=0,
                                )

lgbm.fit(X_train, y_train)
end=time.time() 
print("time: ",end-start) # 9s vs 33 s H2o light gbm

y_pred=lgbm.predict(X_test)
y_pred_probs=lgbm.predict_proba(X_test)

print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print('classification_report \n',classification_report(y_test, y_pred))


[LightGBM] [Warning] num_threads is set=16, n_jobs=-1 will be ignored. Current value: num_threads=16
[LightGBM] [Warning] min_data_in_leaf is set=2000, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2000
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
time:  11.51891303062439
f1 score 0.0
balanced_accuracy_score: 0.5
classification_report 
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     84806
           1       0.00      0.00      0.00      7448

    accuracy                           0.92     92254
   macro avg       0.46      0.50      0.48     92254
weighted avg       0.85      0.92      0.88     92254



In [ ]:
start=time.time()
#clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
#    max_depth=1, random_state=0).fit(X_train, y_train)
clf = RandomForestClassifier().fit(X_train, y_train)
end=time.time() 
print("time: ",end-start)
y_pred=clf.predict(X_test)
print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print('classification_report \n',classification_report(y_test, y_pred))


# Credit card

In [ ]:
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder,OrdinalEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import time
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report,roc_curve, auc,precision_recall_curve,f1_score,balanced_accuracy_score

In [ ]:
file='https://media.githubusercontent.com/media/aapatel09/handson-unsupervised-learning/master/datasets/credit_card_data/credit_card.csv'
data = pd.read_csv(file)

dataX = data.copy().drop(['Class'],axis=1)
dataY = data['Class'].copy()

featuresToScale = dataX.columns
sX =StandardScaler(copy=True)
dataX.loc[:,featuresToScale] = sX.fit_transform(dataX[featuresToScale])

X_train, X_test, y_train, y_test = \
    train_test_split(dataX, dataY, test_size=0.20, \
                    random_state=2018, stratify=dataY)

In [ ]:
start=time.time()
xgb=XGBClassifier(
                nthread=16,
                objective='binary:logistic',
                eval_metric = 'logloss',
                #scale_pos_weight=1,
                seed=2018,
                verbosity=1    
                              )
xgb.fit(X_train, y_train)
end=time.time() 
print("time: ",end-start) # 89 s compare with 45 s h2o r

y_pred=xgb.predict(X_test)

print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print('classification_report \n',classification_report(y_test, y_pred))

/home/dnn/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


time:  89.60068416595459
f1 score 0.88268156424581
balanced_accuracy_score: 0.9030436386710001
classification_report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.98      0.81      0.88        98

    accuracy                           1.00     56962
   macro avg       0.99      0.90      0.94     56962
weighted avg       1.00      1.00      1.00     56962



In [ ]:
start=time.time()
lgbm=LGBMClassifier(
    num_threads=16,
    #n_estimators = 200,
    #boosting_type = 'gbdt',
    #objective = 'binary',
    #metric= 'binary:logloss',
    #metric_freq=50,
    #learning_rate=0.01,
    #max_depth=4,
    #num_leaves= 16, #(2^maxdepth)
    #min_data_in_leaf=2000,
    #scale_pos_weight=1,
    #num_threads=16,
    #random_state =500,
    #bagging_freq=0,
                                )
lgbm.fit(X_train, y_train)
end=time.time() # 4.7 s compare with 7 s h2o r
print("time: ",end-start)

y_pred=lgbm.predict(X_test)
print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print('classification_report \n',classification_report(y_test, y_pred))

[LightGBM] [Warning] num_threads is set=16, n_jobs=-1 will be ignored. Current value: num_threads=16
time:  4.736140489578247
f1 score 0.6285714285714286
balanced_accuracy_score: 0.8363302200452494
classification_report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.59      0.67      0.63        98

    accuracy                           1.00     56962
   macro avg       0.79      0.84      0.81     56962
weighted avg       1.00      1.00      1.00     56962



In [ ]:
start=time.time()
#clf = GradientBoostingClassifier(n_jobs=-1).fit(X_train, y_train)
clf = RandomForestClassifier(n_jobs=-1).fit(X_train, y_train) # 100 tree
end=time.time() 
print("time: ",end-start) # 128 s compare with 81 s h2o r
y_pred=clf.predict(X_test)
print("f1 score",f1_score(y_test,y_pred))
print("balanced_accuracy_score:",balanced_accuracy_score(y_test,y_pred))
print('classification_report \n',classification_report(y_test, y_pred))

time:  128.91356110572815
f1 score 0.8777777777777778
balanced_accuracy_score: 0.9030348457616023
classification_report 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.81      0.88        98

    accuracy                           1.00     56962
   macro avg       0.98      0.90      0.94     56962
weighted avg       1.00      1.00      1.00     56962



# Pipe line & Transformer

https://towardsdatascience.com/pipeline-columntransformer-and-featureunion-explained-f5491f815f

## 1. Pipeline

In [21]:
# Set seed for reproducibility
seed = 123

# Import package/module for data
import numpy as np
import pandas as pd
from seaborn import load_dataset

# Import modules for feature engineering and modelling
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

# Load dataset
df = load_dataset('tips').drop(columns=['tip', 'sex']).sample(n=5, random_state=seed)

# Add missing values
df.iloc[[1, 2, 4], [2, 4]] = np.nan
df

,total_bill,smoker,day,time,size
112,38.07,No,Sun,Dinner,3.0
19,20.65,No,NaN,Dinner,NaN
187,30.46,Yes,NaN,Dinner,NaN
169,10.63,Yes,Sat,Dinner,2.0
31,18.35,No,NaN,Dinner,NaN


In [24]:
df.shape

(5, 5)

In [22]:
# Partition data
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['total_bill', 'size']), 
                                                    df['total_bill'], 
                                                    test_size=.2, 
                                                    random_state=seed)

In [23]:
# Impute training data
imputer = SimpleImputer(strategy='constant', fill_value='missing')
X_train_imputed = imputer.fit_transform(X_train)

# Encode training data
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_encoded = encoder.fit_transform(X_train_imputed)

# Inspect training data before and after
print("******************** Training data ********************")
display(X_train)
display(pd.DataFrame(X_train_imputed, columns=X_train.columns))
display(pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names(X_train.columns)))

# Transform test data
X_test_imputed = imputer.transform(X_test)
X_test_encoded = encoder.transform(X_test_imputed)

# Inspect test data before and after
print("******************** Test data ********************")
display(X_test)
display(pd.DataFrame(X_test_imputed, columns=X_train.columns))
display(pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names(X_train.columns)))

******************** Training data ********************


,smoker,day,time
169,Yes,Sat,Dinner
31,No,NaN,Dinner
112,No,Sun,Dinner
187,Yes,NaN,Dinner


,smoker,day,time
0,Yes,Sat,Dinner
1,No,missing,Dinner
2,No,Sun,Dinner
3,Yes,missing,Dinner


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner
0,0.0,1.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,1.0
2,1.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,1.0


******************** Test data ********************


,smoker,day,time
19,No,NaN,Dinner


,smoker,day,time
0,No,missing,Dinner


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner
0,1.0,0.0,0.0,0.0,1.0,1.0


For each dataset, we see raw data first, then output after imputation and the output after encoding last. This approach gets the job done. However, we manually feed output from previous step to the next step as an input and have multiple interim outputs. We also had to repeat each step on the test data. As the number of steps increase, it will become more tedious to maintain and more prone to error. We can write more streamlined and concise code with Pipeline:

In [25]:
# Fit pipeline to training data
pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                 ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])
pipe.fit(X_train)

# Inspect training data before and after
print("******************** Training data ********************")
display(X_train)
display(pd.DataFrame(pipe.transform(X_train), columns=pipe['encoder'].get_feature_names(X_train.columns)))

# Inspect test data before and after
print("******************** Test data ********************")
display(X_test)
display(pd.DataFrame(pipe.transform(X_test), columns=pipe['encoder'].get_feature_names(X_train.columns)))


******************** Training data ********************


,smoker,day,time
169,Yes,Sat,Dinner
31,No,NaN,Dinner
112,No,Sun,Dinner
187,Yes,NaN,Dinner


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner
0,0.0,1.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,1.0
2,1.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,1.0,1.0


******************** Test data ********************


,smoker,day,time
19,No,NaN,Dinner


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner
0,1.0,0.0,0.0,0.0,1.0,1.0


ADD MODEL:

In [27]:
# Impute training data
imputer = SimpleImputer(strategy='constant', fill_value='missing')
X_train_imputed = imputer.fit_transform(X_train)

# Encode training data
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_encoded = encoder.fit_transform(X_train_imputed)

# Fit model to training data
model = LinearRegression()
model.fit(X_train_encoded, y_train)

# Predict training data
y_train_pred = model.predict(X_train_encoded)
print(f"Predictions on training data: {y_train_pred}")

# Transform test data
X_test_imputed = imputer.transform(X_test)
X_test_encoded = encoder.transform(X_test_imputed)

# Predict test data
y_test_pred = model.predict(X_test_encoded)
print(f"Predictions on test data: {y_test_pred}")

Predictions on training data: [10.63 18.35 38.07 30.46]
Predictions on test data: [18.35]


In [29]:
# Fit pipeline to training data
pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                 ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)), 
                 ('model', LinearRegression())])
pipe.fit(X_train, y_train)

# Predict training data
y_train_pred = pipe.predict(X_train)
print(f"Predictions on training data: {y_train_pred}")

# Predict test data
y_test_pred = pipe.predict(X_test)
print(f"Predictions on test data: {y_test_pred}")

Predictions on training data: [10.63 18.35 38.07 30.46]
Predictions on test data: [18.35]


1. Ability to fine-tune pipeline: When building a model, have you ever had to go back a step to try a different way to preprocess the data and run the model again to see if a tweak in preprocessing step improves the fitness of the model? When optimising a model, the cogs are not only in the model hyperparameters but also in the implementation of preprocessing steps. With this in mind, when we have a single pipeline object that unifies transformers and an estimator, we are able to fine-tune the hyperparameters of the entire pipeline including both transformers and an estimator with either GridSearchCV or RandomizedSearchCV.

2. Easier deployment: All transforming steps used to prepare the data when training a model should also be applied to the data in production environment when making predictions. When we train a Pipeline, we train a single object which contains data transformers and a model. Once trained, this Pipeline object can be used for smoother deployment.

## 2. Column Transformer

In the previous example, we imputed and encoded all columns the same way. However, we often need to apply different sets of tranformers to different groups of columns. For instance, we would want to apply OneHotEncoder to only categorical columns but not to numerical columns. This is where ColumnTransformer comes in. This time, we will partition the dataset keeping all columns so that we have both numerical and categorical features.

In [30]:

# Partition data
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['total_bill']), 
                                                    df['total_bill'], 
                                                    test_size=.2, 
                                                    random_state=seed)

# Define categorical columns
categorical = list(X_train.select_dtypes('category').columns)
print(f"Categorical columns are: {categorical}")

# Define numerical columns
numerical = list(X_train.select_dtypes('number').columns)
print(f"Numerical columns are: {numerical}")

Categorical columns are: ['smoker', 'day', 'time']
Numerical columns are: ['size']


In [32]:
# Define categorical pipeline
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Fit column transformer to training data
preprocessor = ColumnTransformer(transformers=[('cat', cat_pipe, categorical)], 
                                 remainder='passthrough')
preprocessor.fit(X_train)

# Prepare column names
cat_columns = preprocessor.named_transformers_['cat']['encoder'].get_feature_names(categorical)
columns = np.append(cat_columns, numerical)

# Inspect training data before and after
print("******************** Training data ********************")
display(X_train)
display(pd.DataFrame(preprocessor.transform(X_train), columns=columns))

# Inspect test data before and after
print("******************** Test data ********************")
display(X_test)
display(pd.DataFrame(preprocessor.transform(X_test), columns=columns))

******************** Training data ********************


,smoker,day,time,size
169,Yes,Sat,Dinner,2.0
31,No,NaN,Dinner,NaN
112,No,Sun,Dinner,3.0
187,Yes,NaN,Dinner,NaN


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner,size
0,0.0,1.0,1.0,0.0,0.0,1.0,2.0
1,1.0,0.0,0.0,0.0,1.0,1.0,NaN
2,1.0,0.0,0.0,1.0,0.0,1.0,3.0
3,0.0,1.0,0.0,0.0,1.0,1.0,NaN


******************** Test data ********************


,smoker,day,time,size
19,No,NaN,Dinner,NaN


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner,size
0,1.0,0.0,0.0,0.0,1.0,1.0,NaN


In [33]:
# Define categorical pipeline
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', MinMaxScaler())])

# Combine categorical and numerical pipelines
preprocessor = ColumnTransformer(transformers=[('cat', cat_pipe, categorical),
                                               ('num', num_pipe, numerical)])

# Fit a pipeline with transformers and an estimator to the training data
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', LinearRegression())])
pipe.fit(X_train, y_train)

# Predict training data
y_train_pred = pipe.predict(X_train)
print(f"Predictions on training data: {y_train_pred}")

# Predict test data
y_test_pred = pipe.predict(X_test)
print(f"Predictions on test data: {y_test_pred}")

Predictions on training data: [10.63 18.35 38.07 30.46]
Predictions on test data: [18.35]


## 3. Feature Union

In [34]:
# Define custom transformer
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

# Define categorical pipeline
cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical)),
                     ('imputer', SimpleImputer(strategy='median')),
                     ('scaler', MinMaxScaler())])

# Fit feature union to training data
preprocessor = FeatureUnion(transformer_list=[('cat', cat_pipe),
                                              ('num', num_pipe)])
preprocessor.fit(X_train)

# Prepare column names
cat_columns = preprocessor.transformer_list[0][1][2].get_feature_names(categorical)
columns = np.append(cat_columns, numerical)

# Inspect training data before and after
print("******************** Training data ********************")
display(X_train)
display(pd.DataFrame(preprocessor.transform(X_train), columns=columns))

# Inspect test data before and after
print("******************** Test data ********************")
display(X_test)
display(pd.DataFrame(preprocessor.transform(X_test), columns=columns))

******************** Training data ********************


,smoker,day,time,size
169,Yes,Sat,Dinner,2.0
31,No,NaN,Dinner,NaN
112,No,Sun,Dinner,3.0
187,Yes,NaN,Dinner,NaN


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner,size
0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,1.0,1.0,0.5
2,1.0,0.0,0.0,1.0,0.0,1.0,1.0
3,0.0,1.0,0.0,0.0,1.0,1.0,0.5


******************** Test data ********************


,smoker,day,time,size
19,No,NaN,Dinner,NaN


,smoker_No,smoker_Yes,day_Sat,day_Sun,day_missing,time_Dinner,size
0,1.0,0.0,0.0,0.0,1.0,1.0,0.5


In [35]:
# Define categorical pipeline
cat_pipe = Pipeline([('selector', ColumnSelector(categorical)),
                     ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('selector', ColumnSelector(numerical)),
                     ('imputer', SimpleImputer(strategy='median')),
                     ('scaler', MinMaxScaler())])

# Combine categorical and numerical pipeline
preprocessor = FeatureUnion(transformer_list=[('cat', cat_pipe),
                                              ('num', num_pipe)])

# Combine categorical and numerical pipeline
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('model', LinearRegression())])
pipe.fit(X_train, y_train)

# Predict training data
y_train_pred = pipe.predict(X_train)
print(f"Predictions on training data: {y_train_pred}")

# Predict test data
y_test_pred = pipe.predict(X_test)
print(f"Predictions on test data: {y_test_pred}")

Predictions on training data: [10.63 18.35 38.07 30.46]
Predictions on test data: [18.35]


# Check pipeline in real dataset

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import pipeline
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold,train_test_split,cross_val_score,cross_validate
#ML normal ML
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

#ML ensembles
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.metrics import recall_score,precision_score,roc_curve,auc,precision_recall_curve
from sklearn.metrics import f1_score,balanced_accuracy_score,average_precision_score
from sklearn.metrics import make_scorer

# explain
import shap
import dalex as dx

In [2]:
path='https://raw.githubusercontent.com/rstudio/keras-customer-churn/master/data/WA_Fn-UseC_-Telco-Customer-Churn.csv'
df=pd.read_csv(path)
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
target="Churn"
X = df.drop(columns=target)
y= df[target]
print(X.shape)
print(y.shape)
X_train_a, X_test, y_train_a, y_test = model_selection.train_test_split(X,y, test_size=0.2, 
                                    random_state=2018, stratify=y)

X_train, X_val, y_train, y_val = model_selection.train_test_split(X_train_a, y_train_a,test_size=0.2, 
                                    random_state=2018, stratify=y_train_a)

(7043, 20)
(7043,)


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class DropSomeColumns(BaseEstimator, TransformerMixin):

    def __init__(self, cols):
        if not isinstance(cols, list):
            self.cols = [cols]
        else:
            self.cols = cols

    def fit(self, X: pd.DataFrame, y: pd.Series):
        # there is nothing to fit
        return self

    def transform(self, X:pd.DataFrame):
        X = X.copy()
        return X[self.cols]

In [14]:
pipe=Pipeline(DropSomeColumns("abc"))

TypeError: type object argument after * must be an iterable, not DropSomeColumns